In [ ]:
# externals
import os
import pathlib
import platform
import re
import subprocess

In [ ]:
# get the current working directory
cwd = pathlib.PosixPath(os.getcwd())
# show me
print(f"cwd: {cwd}")

In [ ]:
# the name of the app, hence the name of its environment
app = "qed"

# the current working directory
cwd = pathlib.PosixPath(os.getcwd())
# the directory with the support files
support_dir = cwd / "support"

# the user's home directory
home_dir = pathlib.PosixPath("~").expanduser()

# the home of local conda environments
base_dir = home_dir / ".local"
# the home of tools shared by all environments
base_bin_dir = base_dir / "bin"

# the location of the qed environment
qed_dir = base_dir / "envs" / app
# the location of its tools
qed_bin_dir = qed_dir / "bin"
# headers
qed_inc_dir = qed_dir / "include"
# and libraries
qed_lib_dir = qed_dir / "lib"

# the XDG compliant configuration area
xdg_dir = home_dir / ".config"
# the location of the pyre configuration directory
pyre_dir = xdg_dir / "pyre"
# the location with the mm package database
mm_dir = xdg_dir / "mm"

# the location of the source code
# we will clone the various repositories in here
# feel free to rename this to whatever makes sense to you
src_dir = home_dir / "dv"

# the install location of the build products
prefix_dir = qed_dir

# temporary storage for the intermediate build products
bldroot_dir = home_dir / "tmp/builds" / app

In [ ]:
# build a wrapper around {os.system} so we can echo the command before executing it
def shell(command):
    """
    Print a command before executing it
    """
    # echo
    print(f" -- executing: {command}")
    # and do it
    return os.system(command)

# and one over {os.chdir} so we can print the new directory before moving there
def cd(path):
    """
    Print the destination directory before going there
    """
    # echo
    print(f" -- cwd: {path}")
    # and do it
    return os.chdir(path)

In [ ]:
# create the directories we need
# the base environment directory
base_dir.mkdir(parents=True, exist_ok=True)
# and the place with the shared tool
base_bin_dir.mkdir(parents=True, exist_ok=True)
# the pyre configuration directory
pyre_dir.mkdir(parents=True, exist_ok=True)
# the location of the mm package database
mm_dir.mkdir(parents=True, exist_ok=True)
# finally, the source directory
src_dir.mkdir(parents=True, exist_ok=True)

# build a table that maps system names to the ones used by the micromamba server
systems = {
    "Darwin": "osx",
    "Linux": "linux",
}
# and one that maps the architectures
machines = {
    "arm64": "arm64",
    "ppc64le": "ppc64le",
    "x86_64": "64",
}
# interrogate the host
uname = platform.uname()
# extract what we need
system = systems[uname.system]
machine = machines[uname.machine]

# form the path to micromamba
micromamba = base_bin_dir / "micromamba"
# if it doesn't exist
if not micromamba.exists():
    # show me
    print("-- installing micromamba")
    # build the url
    url = f"https://micro.mamba.pm/api/micromamba/{system}-{machine}/latest"
    # fetch micromamba and install it
    shell(f"curl -Ls {url} | tar xvj --directory {base_dir} bin/micromamba")

# mark
os.environ["MAMBA_ROOT_PREFIX"] = str(base_dir)

In [ ]:
# the environment configuration file is machine specific
# mostly because on osx we don't install a compiler
env_file = support_dir / f"env-{system}-{machine}.yaml"

# if the environment does not exist
if not qed_dir.exists():
    # install the environment
    env_command = "install"
# otherwise
else:
    # update it
    env_command = "update"
# the options
env_options = f"--yes --prefix {qed_dir} --file {support_dir / env_file}"

# execute
shell(f"{micromamba} {env_command} {env_options}")


In [ ]:
# build a shortcut for running commands in this environment
run = f"{micromamba} run --prefix {qed_dir}"
# use the shortcut to build the {git} command
git = f"{run} git"


In [ ]:
# find the version of python
# there is no easy way do this...

# set up the command line
argv = [
    micromamba,
    "run",
    "--prefix", str(qed_dir),
    "python3", "--version"
]
# and the options to popen
options = {
    "executable": micromamba,
    "args": argv,
    "stdout": subprocess.PIPE,
    "stderr": subprocess.PIPE,
    "universal_newlines": True,
    "shell": False,
}
# launch the interpreter
with subprocess.Popen(**options) as cmd:
    # collect the output
    stdout, stderror = cmd.communicate()
    # get the status
    status = cmd.returncode
    # if something went wrong
    if status != 0:
        # ooops
        raise RuntimeError("could not retrieve the version of the python interpreter")
    # parse the output
    match = re.match(r"Python (?P<major>\d+)\.(?P<minor>\d+).(?P<micro>\d+)", stdout)
    # if we failed to match
    if not match:
        # ooops
        raise RuntimeError("could not retrieve the version of the python interpreter")
    # unpack
    python_major = match.group("major")
    python_minor = match.group("minor")
    python_micro = match.group("micro")

    # and show me
    print(f" -- python version: {python_major}.{python_minor}.{python_micro}")

In [ ]:
# build the source directory
src_dir.mkdir(parents=True, exist_ok=True)
# get the hdf5 repository with the necessary bug fixes
cd(src_dir)
# we'll remove this once there a release that works
h5_src_dir = src_dir / "hdf5"
# if don't have the source
if not h5_src_dir.exists():
    # show me
    print(" -- cloning hdf5")
    # the url to the h5 repository with support for long AWS authentication tokens
    h5_url = "https://github.com/aivazis/hdf5"
    # clone it
    os.system(f"{git} clone {h5_url}")

# go there
cd(h5_src_dir)
# checkout the correct branch
shell(f"{git} checkout s3")
# refresh
shell(f"{git} pull")
# clean any old builds
shell(f"{git} clean -dfx")
# generate the configuration file
shell(f"{run} ./autogen.sh")
# configure
shell(
    f"CC={qed_bin_dir}/clang CXX={qed_bin_dir}/clang++"
    f" CPPFLAGS=-I'{qed_inc_dir}'"
    f" LDFLAGS=-'L{qed_lib_dir}'"
    f" {run} ./configure --prefix='{qed_dir}' --enable-hl --enable-cxx --enable-ros3-vfd"
)
# build it
shell(f"{run} make -j")
# and install it
shell(f"{run} make -j install")

In [ ]:
# build the source directory
src_dir.mkdir(parents=True, exist_ok=True)
# go to the source directory
cd(src_dir)
# clone or update mm
mm_src_dir = src_dir / "mm"

# if we don't already have the source
if not mm_src_dir.exists():
    # clone it
    shell(f"{git} clone https://github.com/aivazis/mm")
# go there
cd(mm_src_dir)
# refresh
shell(f"{run} git pull")

# the mm configuration file
mm_in_file = support_dir / f"mm-{system}-{machine}.yaml"
# gets installed here
mm_out_file = pyre_dir / "mm.yaml"
# open the output file
with open(mm_out_file, mode="w") as ostream:
    # and the input file
    with open(mm_in_file, mode="r") as istream:
        # read the contents
        contents = istream.read()
        # expand the macros
        contents = contents.replace("@PYTHON_VERSION@", f"{python_major}.{python_minor}")
        # and write them out
        ostream.write(contents)

# the mm package database
pkg_in_file = support_dir / "config.mm"
# gets installed here
pkg_out_file = mm_dir / "config.mm"
# open the output file
with open(pkg_out_file, mode="w") as ostream:
    # and the input file
    with open(pkg_in_file, mode="r") as istream:
        # read the contents
        contents = istream.read()
        # expand the macros
        contents = contents.replace("@PYTHON_VERSION@", f"{python_major}.{python_minor}")
        # and write them out
        ostream.write(contents)

# setup the command for invoking mm
mm = f"{run} python3 {mm_src_dir / 'mm'}"

In [ ]:
# build the source directory
src_dir.mkdir(parents=True, exist_ok=True)
# next up, pyre
cd(src_dir)
# clone or update pyre
pyre_src_dir = src_dir / "pyre"
# if we don't have the source
if not pyre_src_dir.exists():
    # the url to the pyre repository
    pyre_url = "https://github.com/pyre/pyre"
    # clone it
    shell(f"{git} clone {pyre_url}")

# go there
cd(pyre_src_dir)
# refresh
shell(f"{git} pull")

# show me where it will end up
shell(f"{mm} builder.info")
# build it
shell(f"{mm} --slots=24")

In [ ]:
# build the source directory
src_dir.mkdir(parents=True, exist_ok=True)
# next up, qed
cd(src_dir)
# clone or update pyre
qed_src_dir = src_dir / "qed"
# if we don't have the source
if not qed_src_dir.exists():
    # the url to the qed repository
    qed_url = "https://github.com/aivazis/qed"
    # clone it
    os.system(f"{git} clone {qed_url}")

# go there
cd(qed_src_dir)
# refresh
shell(f"{git} pull")

# show me where it will end up
shell(f"{mm} builder.info")
# build it
shell(f"{mm}")